In [1]:
from select import select
from symbol import dotted_as_name
import turtle
import math
import random
import time
from time import sleep
from sys import argv


In [ ]:
class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(1)
    
    def _swap_turn(self,turn):
        if turn == "red":
            return "blue"
        else:
            return "red"
    
                
    def _evaluate(self):
        #TODO
        return 1
    
    def bestMove(self, depth, player_turn):
        bestScore = -float("inf")
        bestmove = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    if (i,j) in self.available_moves:
                        self.red.append((i,j))
                        self.available_moves.remove((i,j))
                        score = self.minimax(1, "red", -float("inf"),float("inf")) #minimizing
                        self.available_moves.append((i,j))
                        self.red.remove((i,j))
                        if score > bestScore:
                            bestScore = score
                            bestmove = (i,j)
                        if score == bestScore:
                            if random.random() < 0.5:
                                bestmove = (i,j)
                        


                       
                        
        return bestmove


    def minimax(self, depth, player_turn, alpha, beta):
        #TODO
        result = self.gameover(self.red,self.blue)
        score = depth * (10)
        if result != 0:
            if result == "blue": score += -100
            elif result == "red": score += 100
            return score

        
        
        if depth >= self.minimax_depth:
            return score
        
        if player_turn == "red": #maximizing
            bestscore = -float("inf")
            for i in range(0, 6):
                for j in range(i, 6):
                    if i != j:
                        if (i,j) in self.available_moves:
                            self.red.append((i,j))
                            self.available_moves.remove((i,j))
                            score = self.minimax(depth +1 , "blue",alpha, beta) #minimizing
                            self.available_moves.append((i,j))
                            self.red.remove((i,j))
                            bestscore = max(score,bestscore)
                            alpha = max(alpha, bestscore)
                            if beta <= alpha and self.prune == True:
                                break
                if beta <= alpha:
                    break
            return bestscore
        
        elif player_turn == "blue": #minimizing
            bestscore = float("inf")
            for i in range(0, 6):
                for j in range(i, 6):
                    if i != j:
                        if (i,j) in self.available_moves:
                            self.blue.append((i,j))
                            self.available_moves.remove((i,j))
                            score = self.minimax(depth +1 , "red",alpha, beta) #maximizing
                            self.available_moves.append((i,j))
                            self.blue.remove((i,j))
                            bestscore = min(score,bestscore)
                            beta = min(beta , bestscore)
                            if (beta <= alpha)  and (self.prune == True):
                                break
                if beta <= alpha:
                    break
            return bestscore
        
                            
        
    def enemy(self):
        return random.choice(self.available_moves)

    def play(self):
        self.initialize()
        while True:
            if self.turn == 'red':
                #selection = self.minimax(depth=self.minimax_depth, player_turn=self.turn)
                selection = self.bestMove(depth=self.minimax_depth, player_turn=self.turn)
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0

In [ ]:
game = Sim(minimax_depth=1, prune= False, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(200):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=3, prune= False, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(200):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=5, prune= False, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(1):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=1, prune= True, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(200):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=3, prune= True, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(200):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=5, prune= True, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(1):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

In [ ]:
game = Sim(minimax_depth=7, prune= True, gui=0)

results = {"red": 0, "blue": 0}
start = time.time()
for i in range(200):
    #print(i)
    results[game.play()] += 1
end = time.time()
print("result:")   
print(results)
print("elapsed time:" + str(end -start))

1. a good heuristic must lead to winning conditions from every path.
The heuristic used in this algorithm gives winning condition +100 scores and loosing condition -100 score. this is done independant of the graphical implementation of the winning condition.
since the opponent acts randomly a positive point (+15) is also given to the length of path, which means the longer the game goes the more probable for the opponent is to make mistake. But this score must be capped at winning condition casuse it may lead to choosing this score over the immadiate win.

2. The higher the depth of the algorithm is, the more persice actions it will take, so as the results show, by taking more steps the winning rate also increases:
1 step: 87%
3 steps: 89%
5 steps: 92%
7 steps: 97%

the downside of increasing the depth is that the number of calculations also increases by a factorial rate
for example for 1 step if we take there is 15 moves so it traverses 15 nodes: (0.02s);
3 steps take 15.14.13 operatins(and traversed nodes) to calculate the minimax value (0.6s);
5 steps take 15.14.13.12.11 operations (8.2s);
7 steps take 15.14.13.12.11.10.9 operations (88.39s);

since pruning is applied, the time difference is not proportional to the number of operations

3. In this implementation the nodes are added by the numerical order ((0,0),(0,1),(0,2), ... ) since the winning condition is randomly appeared by the decisions of the opponent it is not important to have a certain pattern of adding the nodes; the only move which should be fully traversed is the first child and after that pruning comes in and handles the rest of the nodes.
although it is not that important, starting with the nodes which are probable to be less than beta and more than alpha may help the pruning
but the code written in this project aims to keep the simplicity of implementation.